In [75]:
import random
import numpy as np
from MLP_class import *

In [76]:
import matplotlib.pyplot as plt
# torch is just for the feature extractor and the dataset (NOT FOR IMPLEMENTING NEURAL NETWORKS!)
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.models import resnet34
import torch.nn as nn
# sklearn is just for evaluation (NOT FOR IMPLEMENTING NEURAL NETWORKS!)
from sklearn.metrics import confusion_matrix, f1_score

ModuleNotFoundError: No module named 'torch'

In [58]:


weight_input = "weight input"
weight_hidden = "weight hidden"
weight_output = "weight output"
bias_hidden = "bias hidden"
bias_output = "bias output"
fitness = "fitness"
weight_input_hidden = "weight input hidden"
weight_hidden_output = "weight hidden output"
num_of_chromosomes = 50
num_of_generations = 20


Max_weight = 1.199879380338843    # 2 * 0.5999396901694214
Min_weight = -0.7427780198641212   # 2 * -0.3713890099320606

Max_bias = 0.22740982998   # 2 * 0.11370491499578113
Min_bias = -0.5411721827397528  # 2 * -0.2705860913698764

crossover_rate = 0.8

In [63]:
def create_chromosomes(num_chromosomes):
    chromosomes = []
    for i in range(num_chromosomes):
        # a = [i for i in ]
        chro = dict()
        chro[weight_input_hidden] = [[random.uniform(Min_weight, Max_weight) for j in range(20)] for i in range(512)]
        # chro[weight_hidden] = [random.uniform(Min_weight, Max_weight) for _ in range(20)]
        # chro[weight_output] = [random.uniform(Min_weight, Max_weight) for _ in range(10)]
        chro[weight_hidden_output] = [[random.uniform(Min_weight, Max_weight) for j in range(10)] for i in range(20)]
        chro[bias_hidden] = [random.uniform(Min_bias, Max_bias) for _ in range(20)]
        chro[bias_output] = [random.uniform(Min_bias, Max_bias) for _ in range(10)]
        chro[fitness] = 0.0
        chromosomes.append(chro)

    return chromosomes


In [64]:
print(create_chromosomes(2))

[{'weight input hidden': [[1.1594950638256165, 0.4531770045432606, 0.6884023651110811, 0.7465877444998079, 0.8997238918471098, 0.03183687036417282, 0.22524937081638907, -0.43885759450853895, -0.14284732370056452, 0.03271587768792694, 1.0966095215548288, 0.9767041243160397, -0.3174482984651934, -0.7103594589822444, -0.659337476429109, 0.34976519526143734, 0.6261294447081012, 0.5978260639265556, -0.2974568931049728, -0.10194271744753158], [0.9744417733716828, 0.9330206843377351, 0.5237888157147539, 0.25332120672704606, 0.5076978955797276, 0.7763929223931931, 0.383686962278315, 1.0586855856096729, -0.0783484334557164, 0.9198878173542457, 0.4092452753600955, 1.1238084676123363, 0.469133609028632, -0.4896173810480593, 1.058387311983732, -0.170486719969418, 0.3990265784528535, 0.6770650922813692, -0.08250223741202012, 1.076974658856087], [1.1548648247920492, 0.8767337936462718, -0.642235594740286, 0.5287409742450102, 0.5395504471979322, 1.0088715029425688, 0.13204096144417132, -0.17588654274

In [72]:
chro = create_chromosomes(2)
print(chro[0][weight_input_hidden][0][0])
print(chro[1][weight_input_hidden][0])

-0.6902362466905225
[0.9588290930699188, 0.9899289473054405, -0.2978245357054966, 0.4575652493087683, 0.48130505797239276, 0.8386225483006478, 0.2714282673856725, -0.6851477860284104, 0.6944190405952364, 0.11091366845890172, -0.728473930439707, 0.7458320281103705, -0.689505383412499, 0.8105493605922094, 1.0275547782772427, -0.7253340352748907, 0.3208388293077937, 0.8895743736896355, 0.6455237270961081, -0.3628439902976819]


In [70]:
def best_crossovers(parent1, parent2):
    # return parent2, parent1
    
    crossover_point = int(random.uniform(0, 1) * len(parent1))
    # print(crossover_point)
    p = random.randint(0, 1)
    if p == 0:
        offspring1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        offspring2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    else:
        offspring1 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
        offspring2 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    
    return offspring1.tolist(), offspring2.tolist()

In [49]:
def AVG_crossover(parent1, parent2):
    crossover_point = int(random.uniform(0, 1) * len(parent1))
    for i in range(len(parent1)):
        parent1[i] = (parent1[i] + parent2[i]) / 2
        parent2[i] = parent1[i]
    return parent1, parent2
    

In [71]:
# crossover weight input
offspring1_weight_input_hidden, offspring2_weight_input_hidden = best_crossovers(chro[0][weight_input_hidden][0], chro[1][weight_input_hidden][0])
# crossover weight hidden
offspring1_weight_hidden_output , offspring2_weight_hidden_output = best_crossovers(chro[0][weight_hidden_output][0], chro[1][weight_hidden_output][0])
# crossover weight output
# offspring1_weight_output, offspring2_output = best_crossovers(chro[0][weight_output], chro[1][weight_output])


offspring1_bias_hidden, offspring2_bias_hidden = best_crossovers(chro[0][bias_hidden], chro[1][bias_hidden])
offspring1_bias_output, offspring2_bias_output = best_crossovers(chro[0][bias_output], chro[1][bias_output])

In [ ]:
# # crossover weight input AVG
# offspring1_weight_input_A, offspring2_input = best_crossovers(chro[0][weight_input], chro[1][weight_input])
# # crossover weight hidden
# offspring1_weight_hidden , offspring2_hidden = best_crossovers(chro[0][weight_hidden], chro[1][weight_hidden])
# # crossover weight output
# offspring1_weight_output, offspring2_output = best_crossovers(chro[0][weight_output], chro[1][weight_output])


# offspring1_bias_hidden, offspring2_bias_hidden = best_crossovers(chro[0][bias_hidden], chro[1][bias_hidden])
# offspring1_bias_output, offspring2_bias_output = best_crossovers(chro[0][bias_output], chro[1][bias_output])

In [56]:
def mutate(parents, max, min):
    
    select_weight = random.rand(0, len(parent))
    parent[select_weight] = random.uniform(min, max)
    return parent

In [ ]:
def crossover(parents):
    offspring = list()
    parents.sort(key=lambda x: x[fitness], reverse=True)
    for i in range(int(crossover_rate * len(parents) / 2)):
        offspring1, offspring2 = best_crossovers(parents[2 * i], parents[2 * i + 1])
        offspring.append(offspring1)
        offspring.append(offspring2)
        
    return offspring

In [ ]:
def genetic_algorithm():
    # Initialize population
    population = create_chromosomes(num_of_chromosomes)

    # Evaluate population fitness
    # fitness = accuracy of the model 
    evaluate_fitness_population(population)  ## ممد

    # Sort population
    population.sort(key = lambda x: x[fitness])

    print(population)

    # While termination condition is not met
    for _ in range(num_of_generations):
        # Select parents

        # Apply crossover / mutation operators
        offspring = crossover(population)
        mutate(offspring)

        # Evaluate offspring
        evaluate_fitness_population(offspring)

        # Select survivors
        population = select_survivors(population, offspring)

        # Sort population
        population.sort(key = lambda x: x[fitness])

        print(population)
        print(max(population, key=lambda x: x[fitness]))
        mean = sum([x[fitness] for x in population])/len(population)
        chart_fitness.append(mean)

    chart()
    chart_fitness.clear()
    
    return max(population, key=lambda x: x[fitness])   # Return best individual in population

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
feature_extractor = resnet34(pretrained=True)
input_dim = feature_extractor.fc.in_features
for param in feature_extractor.parameters():
  param.requires_grad = False

feature_extractor.fc = nn.Identity()

In [ ]:
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
embeddings = []
labels = []
i = 0
for x, y in train_loader:
  if i == 50:
    break
  i += 1
  print(i)
  embeddings += feature_extractor(x)
  labels += y

In [ ]:
# data_tensor = torch.cat(embeddings, dim=0)

# Convert the tensor to a NumPy array
X = np.array(embeddings)
# data_tensor = torch.cat(labels, dim=0)
for i in range(len(X)):
  X[i] = np.array(X[i])
# Convert the tensor to a NumPy array
Y = np.array(labels)

In [ ]:
result = []
for i in X:
  result.append(i)
result = np.array(result)
result.shape

In [ ]:
mlp = MultiClassMLP()
activations, probs = mlp._forward(result)
